In [46]:
import math

x1, y1 = 0, 0
x2, y2 = 1, 1

dx = x2 - x1
dy = y2 - y1

def get_degrees(dx, dy):
    return math.degrees(math.atan2(dy, dx))

def get_hypo(a, b):
    return math.sqrt(a**2 + b**2)

def get_cos_dgrees(a, b, c):
    return math.degrees(math.acos((a**2 + b**2 - c ** 2) / (2 * a * b)))

r = get_hypo(dx, dy)
print(r)
print(get_degrees(dx, dy))
print(get_cos_dgrees(r, dx, dy))

1.4142135623730951
45.0
44.999999999999986


In [ ]:
a = 6
b = 4
r = get_hypo(a, b)

print(get_cos_dgrees(a/2, get_hypo(3, 4), r))

126.86989764584402


In [202]:
import math 
from collections import *
class PoolTable:
    # 당구판 객체 생성
    def __init__(self, width = 254.0, height = 127.0, R = 5.73):
        self.W = width
        self.H = height
        # 상하좌우에서 임계값 범위 이상으로 넘어갈 수 없음
        # 즉 공의 반지름 이상으로 중앙의 좌표가 넘어갈 수 없다.
        self.threshold = R / 2

    def get_threshold(self):
        # 상 하 좌 우 임계값
        # 공의 가운데 좌표가 넘어갈 수 없는 값
        return (
            self.H - self.threshold,
            self.threshold,
            self.threshold,
            self.W - self.threshold,
        )

    def get_holes(self):
        # 홀의 위치
        # 좌하, 우하, 좌상, 우상, 중하, 중상
        return (
            (0, 0), 
            (self.W, 0), 
            (0, self.H),
            (self.W, self.H), 
            (self.W / 2,0),
            (self.W / 2, self.H), 
            )
    
    def reversed_hole(self, hole, d):
        x차이 = self.W - hole[0]
        y차이 = self.H - hole[1]
        if d == 0:
            # 상단
            res_x = hole[0]
            res_y = hole[1] + y차이
        elif d == 1:
            # 하단
            res_x = hole[0]
            res_y = hole[1] - y차이 
        elif d == 2:
            # 좌측
            res_x = hole[0] - x차이 
            res_y = hole[1]
        else:
            # 좌측
            res_x = hole[0] + x차이 
            res_y = hole[1]
        return res_x, res_y


class Ball:
    # 공 객체 생성
    def __init__(self, num, x, y, R = 5.73):
        self.num = num  # 공의 번호
        # 공의 좌표
        self.x = x
        self.y = y
        # 공의 지름, 반지름
        self.R = R
        self.r = R / 2

    def get_loc(self):
        # 공 위치 리턴
        return self.x, self.y

    def reversed_ball(self, d, table:PoolTable):
        threshold = table.get_threshold()
        x차이 = threshold[d] - self.x
        y차이 = threshold[d] - self.y
        print(x차이, y차이, d, threshold[d])
        if d == 0:
            # 상단
            res_x = self.x
            res_y = self.y + y차이 * 2
        elif d == 1:
            # 하단
            res_x = self.x
            res_y = self.y - y차이 * 2
        elif d == 2:
            # 좌측
            res_x = self.x - x차이 * 2
            res_y = self.y
        else:
            # 우측
            res_x = self.x + x차이 * 2
            res_y = self.y
        return res_x, res_y


def pythagoras(a, b):
    # 피타고라스 정리 / 90도와 마주보는 빗변의 길이
    return (a ** 2 + b ** 2) ** .5

def get_theta(a, b, c):
    # 변의 길이 a, b, c가 있을 때
    # 변 a와 마주보는 각도를 출력
    cosA = (b ** 2 + c ** 2 - a ** 2) / (2 * b * c)
    radA = math.acos(cosA)
    A = math.degrees(radA)
    return A

def get_length_of_line(A, b, c):
    # 각도 A와 붙어있는 선분 b, c의 길이가 주어질 때
    # 각도 A와 마주보는 선분 a의 길이를 출력
    radA = math.radians(A)
    cosA = math.cos(radA)
    a = (b ** 2 + c ** 2 - 2 * b * c * cosA) ** .5
    return a

def get_quadrant(sx, sy, tx, ty):
    # 4분면 검사
    if sx < tx and sy < ty:
        # s의 위치보다 큰 경우
        return 1
    elif sx < tx and sy > ty:
        # s의 위치보다 y좌표가 작은 경우
        return 2
    elif sx > tx and sy > ty:
        # s의 위치보다 모든 좌표가 작은 경우
        return 3
    elif sx > tx and sy < ty:
        # s의 위치보다 y좌표는 크고, x좌표는 작은 경우
        return 4
    elif sx == tx and sy < ty:
        return 1
    elif sx == tx and sy > ty:
        return 3
    elif sx < tx and sy == ty:
        return 1
    elif sx > tx and sy == ty:
        return 3
    else:
        return False


def get_ball_angle(dx, dy):
    return math.degrees(math.atan2(dy, dx))


def straight_shot(cue_ball:Ball, target_ball:Ball, hole):
    cx, cy = cue_ball.get_loc() # 흰공 위치
    tx, ty = target_ball.get_loc() # 타겟공 위치
    hx, hy = hole # 홀 위치
    # 현재 공을 기준으로 타겟과 홀이
    # 같은 사분면에 있는지 확인후
    # 모든 사분면의 좌표를 1사분면에 위치하도록 변경
    ct_quadrant = get_quadrant(cx, cy, tx, ty)
    if not ct_quadrant:
        return False
    elif ct_quadrant != get_quadrant(cx, cy, hx, hy):
        # 같은 사분면 안에 없는 경우
        return False
    elif ct_quadrant == 2:
        ty += (cy - ty) * 2
        hy += (cy - hy) * 2
    elif ct_quadrant == 3:
        tx += (cx - tx) * 2
        hx += (cx - hx) * 2
        ty += (cy - ty) * 2
        hy += (cy - hy) * 2
    elif ct_quadrant == 4:
        tx += (cx - tx) * 2
        hx += (cx - hx) * 2

    # 중간 지점에 넣을 수 없는 경우
    if hx < tx:
        return False
    if cx == tx and tx != hx:
        return False

    # R = 공의 지름
    R = cue_ball.R
    # abx = a 에서 b까지의 x축 거리
    # aby = a 에서 b까지의 y축 거리
    # 홀(h), 흰공(c), 타겟공(t)
    hcx = hx - cx
    hcy = hy - cy
    htx = hx - tx
    hty = hy - ty
    tcx = tx - cx
    tcy = ty - cy
    # a = 흰공에서 홀까지의 거리
    a = pythagoras(hcx, hcy)
    # b = 타겟에서 홀까지의 거리
    b = pythagoras(htx, hty)
    # c = 흰공에서 타겟까지의 거리
    c = pythagoras(tcx, tcy)
    # d = 흰공의 충돌지점에서 홀까지의 거리
    d = b + R

    # 1사분면 혹은 2사분면이며,
    # 공과 홀, 타겟이 모두 같은 선상에 있는 경우
    if hx == cx:
        cue_angle = 0
        if ct_quadrant % 2 == 0:
            cue_angle = 180
        return cue_angle, cue_angle, b + c, (tx, ty)

    # 기준 비율
    stand_ratio = hcy / hcx
    # 타겟 비율
    target_ratio = tcy / tcx

    if stand_ratio == target_ratio:
        # 직선상에 존재한다.
        cue_angle = get_ball_angle(tcx, tcy)
        if ct_quadrant % 2:
            cue_angle = ct_quadrant * 90 - cue_angle
        else:
            cue_angle += (ct_quadrant - 1) * 90
        return cue_angle, cue_angle, b + c, (tx, ty)

    # C = 흰공, 흰공의 충돌지점, 홀 세 지점을 연결 하는 삼각형의 홀 부분의 각도
    C = get_theta(c, a, b)
    # e = 흰공에서 흰공의 충돌지점 까지의 거리
    e = get_length_of_line(C, a, d)

    # D = 흰공, 흰공의 충돌지점, 홀 세 지점을 연결 하는 삼각형의 흰공 부분의 각도
    D = get_theta(d, a, e)
    # X = 흰공 부터 홀까지의 각도
    X = get_theta(hcx, hcy, a)

    def get_goal_loc(e, cue_angle, cue_ball):
        alpha = 90 - cue_angle
        rad_alpha = math.radians(alpha)
        gx = math.cos(rad_alpha) * e + cue_ball.x
        gy = math.sin(rad_alpha) * e + cue_ball.y
        return gx, gy

    # ball_angle = 타겟공 부터 홀까지의 각도
    ball_angle = get_theta(htx, hty, b)
    if stand_ratio > target_ratio:
        # 흰공과 홀까지를 선분으로 이었을 때,
        # 아래쪽에 있는 경우
        cue_angle = X + D
    else:
        # 위쪽에 있는 경우
        cue_angle = X - D
    
    gx, gy = get_goal_loc(e, cue_angle, cue_ball)
    if ct_quadrant == 2:
        gy -= (gy - cy) * 2
    elif ct_quadrant == 3:
        gy -= (gy - cy) * 2
        gx -= (gx - cx) * 2
    elif ct_quadrant == 4:
        gx -= (gx - cx) * 2

    if ct_quadrant % 2: # 1, 3 사분면
        cue_angle += (ct_quadrant - 1) * 90
        ball_angle += (ct_quadrant - 1) * 90
    else: # 2, 4 사분면
        cue_angle = ct_quadrant * 90 - cue_angle
        ball_angle = ct_quadrant * 90 - ball_angle

    return cue_angle, ball_angle, b + c, (gx, gy)


def cal_dist(x1, y1, x2, y2, a, b, R):
    area = abs((x1*y2 + x2*b + a*y1) - (x1*b + a*y2 + x2*y1))
    AB = ((x1 - x2) ** 2 + (y1-y2) ** 2) ** .5
    AC = ((x1 - a) ** 2 + (y1 - b) ** 2) ** .5
    BC = ((x2 - a) ** 2 + (y2 - b) ** 2) ** .5
    dist = area / AB
    min_x = min(x1, x2) - R
    max_x = max(x1, x2) + R
    min_y = min(y1, y2) - R
    max_y = max(y1, y2) + R
    
    if min_x <= a <= max_x and min_y <= b <= max_y:
        return dist
    else:
        return min(AC, BC)

def check_balls(start_ball, goal_loc, other_balls):
    R = start_ball.R
    sx, sy = start_ball.get_loc()
    gx, gy = goal_loc
    for other_ball in other_balls:
        dist = cal_dist(sx, sy, gx, gy, other_ball.x, other_ball.y, start_ball.r)
        if dist < R:
            return False
    return True


def check_balls_oooo(start_ball, goal_loc, other_balls):
    R = start_ball.R
    sx, sy = start_ball.get_loc()
    gx, gy = goal_loc
    for ox, oy in other_balls:
        dist = cal_dist(sx, sy, gx, gy, ox, oy, start_ball.r)
        if dist < R:
            return False
    return True


def one_cution(cue_ball, target_ball:Ball, other_balls, hole):
    res = []
    table = PoolTable()
    for d in range(4):
        r_tx, r_ty = target_ball.reversed_ball(d, table)
        tmp_target_ball = Ball(target_ball.num, r_tx, r_ty)
        r_hole = table.reversed_hole(hole, d)
        print(f"변경된 타겟볼 좌표 {tmp_target_ball.get_loc()}, 변경된 홀 좌표 : {r_hole}, 방향 : {d}")
        result = straight_shot(cue_ball, tmp_target_ball, r_hole)
        if result == False:
            continue
        cue_angle, ball_angle, tot_dist, goals = result
        r_other_balls = list(map(Ball.reversed_ball, other_balls))
        print(r_other_balls)
        if check_balls_oooo(cue_ball, goals, r_other_balls) and check_balls(tmp_target_ball, r_hole, r_other_balls):
            res.append((cue_angle, ball_angle, tot_dist, goals))
    return res

In [203]:
# 테스트 코드
# data_str = "64/64/190/64/-1/-1/198/78/-1/-1/198.5/63.5/"
data_str = "64/64/250/122/-1/-1/-1/-1/-1/-1/-1/-1/"
# data_str = "64/64/250/122/120/8/6/6/173/45/127/64/"
data = list(map(float, data_str[:-1].split('/')))


cue_ball = Ball(0, data[0], data[1])
target_balls = []
# target_locs = [(250, 122), (120, 8), (6, 6), (173, 45), (127, 64)]
for i in range(2, len(data), 2):
    if data[i] < 0 or data[i + 1] < 0:
        continue
    target_balls.append(Ball(i // 2 if i // 2 < 5 else 8, data[i], data[i+1]))


table = PoolTable()
table_map = [[" "] * (int(table.W) // 10 + 1) for _ in range(int(table.H) // 10 + 1)]
print(f"cue_ball : {cue_ball.x}, {cue_ball.y}")

table_map[round(cue_ball.y / 10)][round(cue_ball.x / 10)] = "W"
for target_ball in target_balls:
    table_map[round(target_ball.y / 10)][round(target_ball.x / 10)] = target_ball.num
print("H------------------------H------------------------H")
for row in table_map[1:][::-1]:
    print("|", end="")
    print(*row[1:], end="")
    print("|")
print("H------------------------H------------------------H")
holes = table.get_holes()

other_balls = deque(target_balls)

for _ in range(len(target_balls)):
    target_ball = other_balls.popleft()
    print(f"\n\nball number : {target_ball.num}")
    for hole in holes:
        result = straight_shot(cue_ball, target_ball, hole)
        result2 = one_cution(cue_ball, target_ball, other_balls, hole)
        print("result2 :", result2)
        if result == False:
            continue

        print(f"친공 : {target_ball.x}, {target_ball.y} | 결과 : {result}")
        cue_angle, ball_angle, tot_dist, goals = result

        
        if not check_balls(cue_ball, goals, other_balls):
            print("흰공이 지나가는 길에 다른 공이 존재합니다.")
            continue
        
        if not check_balls(target_ball, hole, other_balls):
            print(f"{target_ball.num}번 공이 지나가는 길에 다른 공이 존재합니다.")
            continue
        
        print(f"{target_ball.num}번 공을 {cue_angle} 각도로 치면 들어갑니다!")
        
    other_balls.append(target_ball)

cue_ball : 64.0, 64.0
H------------------------H------------------------H
|                                                1|
|                                                 |
|                                                 |
|                                                 |
|                                                 |
|                                                 |
|          W                                      |
|                                                 |
|                                                 |
|                                                 |
|                                                 |
|                                                 |
H------------------------H------------------------H


ball number : 1
-125.865 2.135000000000005 0 124.135
변경된 타겟볼 좌표 (250.0, 126.27000000000001), 변경된 홀 좌표 : (0, 127.0), 방향 : 0
-247.135 -119.135 1 2.865
변경된 타겟볼 좌표 (250.0, 360.27), 변경된 홀 좌표 : (0, -127.0), 방향 : 1
-247.135 -119.135 2 2.865
변경된 타겟볼 좌표 (74